In [ ]:
def check_gpu():
    if tf.test.gpu_device_name() != '':
        print("GPU successfully connected.")
    else:
        print("Please connect a GPU.")

%pip install transformers datasets accelerate bitsandbytes pyarrow

from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import torch
import tensorflow as tf

GPU = tf.config.list_physical_devices('GPU')
if GPU:
    print("GPU is available.")
else:
    print("GPU is not available.")

check_gpu()


# **#Task 2**

Data Preprocessing : 1. Reformat the dataset into Llama 2 chat model template
<br>
                     2. Push the reformated data to Hugging Face Hub ( This dataset will be used further)

Following is the prompt template of Llama 2 chat model

![image.png](attachment:763412bc-6f1a-4587-8c62-31931f611a26.png)


In [ ]:
#Here goes the code for data preprocessing
!pip install -q datasets
!pip install -q datasets huggingface_hub

from huggingface_hub import notebook_login
notebook_login()


In [ ]:
from datasets import load_dataset
import re
dataset = load_dataset('jizzu/llama2_indian_law_v2')

dataset = dataset['train']

def transform_conversation(example):
    conversation_text = example['text']
    segments = conversation_text.split('###')

    reformatted_segments = []

    for i in range(1, len(segments) - 1, 2):
        human_text = segments[i].strip().replace('Human:', '').strip()

        if i + 1 < len(segments):
            assistant_text = segments[i+1].strip().replace('Assistant:', '').strip()

            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] {assistant_text} </s>')
        else:
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] </s>')

    return {'text': ''.join(reformatted_segments)}

transformed_dataset = dataset.map(transform_conversation)


In [ ]:
transformed_dataset.push_to_hub("newdata")